In [2]:
# Loading the YOLO

from ultralytics import YOLO

In [2]:
# Briefly displaying architecture

model = YOLO("yolov8n.yaml")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [ ]:
# Training the model for 100 epochs, training is halted at nearly 70th epoch

model.train(data="data/data.yaml", epochs=100, batch=8) 

In [ ]:
# Running inference on a given video

import os
import cv2
video_path = "Highway_cars.mp4"
cap = cv2.VideoCapture("Highway_cars.mp4")

ret, frame = cap.read()
H, W, _ = frame.shape
video_path_out = '{}_output.mp4'.format(video_path)
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

model_path = os.path.join('.', 'runs', 'colab_runs', 'runs', 'detect', 'train2','weights','best.pt')
model = YOLO(model_path)

threshold = 0.3
while ret:
    results = model(frame)[0]
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 255), 2)
            cv2.putText(frame, f"{score:.2f}% {results.names[int(class_id)].lower()}", (int(x1), int(y1 - 8)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
# Impoting libraries for reading and visualizing metrics data

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reading the metrics metadata

df = pd.read_csv("runs/detect/train/results.csv")
df

In [ ]:
# Visualizing metrics

%matplotlib inline
fig, axes = plt.subplots(nrows=len(df.columns), ncols=1, figsize=(10, 6 * len(df.columns)))
for i, col in enumerate(df.columns):
    axes[i].plot(df[col], marker='o', linestyle='-', color='b')
    axes[i].set_title(f'{col}', fontsize=16, fontweight='bold')
    axes[i].set_xlabel('Index', fontsize=12)
    axes[i].set_ylabel(col, fontsize=12)
    axes[i].grid(True)
    
    axes[i].tick_params(axis='both', which='major', labelsize=10)
    axes[i].axhline(y=df[col].mean(), color='r', linestyle='--', label='Mean')
    axes[i].legend(loc='best')

plt.tight_layout()
plt.show()